In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [2]:
files = ['ieee-fraud-detection/test_identity.csv', 
         'ieee-fraud-detection/test_transaction.csv',
         'ieee-fraud-detection/train_identity.csv',
         'ieee-fraud-detection/train_transaction.csv',
         'ieee-fraud-detection/sample_submission.csv']

In [3]:
%%time
def load_data(file):
    return pd.read_csv(file)

with multiprocessing.Pool() as pool:
    test_id, test_tr, train_id, train_tr, sub = pool.map(load_data, files)

CPU times: user 3.68 s, sys: 9.48 s, total: 13.2 s
Wall time: 46.5 s


In [4]:
train = pd.merge(train_tr, train_id, on='TransactionID', how='left')
test = pd.merge(test_tr, test_id, on='TransactionID', how='left')

del test_id, test_tr, train_id, train_tr
gc.collect()

39

In [6]:
useful_features = ['TransactionAmt',
'ProductCD',
'card1',
'card2',
'card3',
'card4',
'card5',
'card6',
'addr1',
'dist1',
'P_emaildomain',
'R_emaildomain',
'C1',
'C2',
'C4',
'C5',
'C6',
'C7',
'C8',
'C9',
'C10',
'C11',
'C12',
'C13',
'C14',
'D1',
'D2',
'D3',
'D4',
'D5',
'D6',
'D8',
'D9',
'D10',
'D11',
'D12',
'D13',
'D14',
'D15',
'M2',
'M3',
'M4',
'M5',
'M6',
'M8',
'M9',
'V4',
'V5',
'V12',
'V13',
'V19',
'V20',
'V30',
'V34',
'V35',
'V36',
'V37',
'V38',
'V44',
'V45',
'V47',
'V49',
'V52',
'V53',
'V54',
'V56',
'V61',
'V62',
'V70',
'V74',
'V75',
'V76',
'V78',
'V81',
'V82',
'V83',
'V87',
'V91',
'V96',
'V97',
'V99',
'V126',
'V127',
'V128',
'V130',
'V131',
'V139',
'V140',
'V143',
'V160',
'V165',
'V170',
'V171',
'V187',
'V189',
'V201',
'V203',
'V204',
'V206',
'V207',
'V208',
'V209',
'V210',
'V213',
'V215',
'V217',
'V222',
'V234',
'V257',
'V258',
'V261',
'V264',
'V265',
'V266',
'V267',
'V268',
'V271',
'V274',
'V275',
'V277',
'V279',
'V280',
'V282',
'V283',
'V285',
'V287',
'V289',
'V291',
'V292',
'V294',
'V306',
'V307',
'V308',
'V310',
'V312',
'V313',
'V314',
'V315',
'V317',
'V323',
'V324',
'V332',
'V333',
'id_01',
'id_02',
'id_03',
'id_05',
'id_06',
'id_13',
'id_14',
'id_17',
'id_19',
'id_20',
'id_32',
'DeviceType',
'DeviceInfo']

In [7]:
cols_to_drop = [col for col in train.columns if col not in useful_features]
cols_to_drop.remove('isFraud')
cols_to_drop.remove('TransactionID')
cols_to_drop.remove('TransactionDT')

In [8]:
print('{} features are going to be dropped for being useless'.format(len(cols_to_drop)))

train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

275 features are going to be dropped for being useless


KeyError: "['id_04' 'id_07' 'id_08' 'id_09' 'id_10' 'id_11' 'id_12' 'id_15' 'id_16'\n 'id_18' 'id_21' 'id_22' 'id_23' 'id_24' 'id_25' 'id_26' 'id_27' 'id_28'\n 'id_29' 'id_30' 'id_31' 'id_33' 'id_34' 'id_35' 'id_36' 'id_37' 'id_38'] not found in axis"

In [9]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Data columns (total 159 columns):
TransactionID     int64
isFraud           int64
TransactionDT     int64
TransactionAmt    float64
ProductCD         object
card1             int64
card2             float64
card3             float64
card4             object
card5             float64
card6             object
addr1             float64
dist1             float64
P_emaildomain     object
R_emaildomain     object
C1                float64
C2                float64
C4                float64
C5                float64
C6                float64
C7                float64
C8                float64
C9                float64
C10               float64
C11               float64
C12               float64
C13               float64
C14               float64
D1                float64
D2                float64
D3                float64
D4                float64
D5                float64
D6                float64
D8               

In [10]:
assholes = ['id_04','id_07','id_08','id_09','id_10','id_11','id_12','id_15' 'id_16'\n 'id_18' 'id_21' 'id_22' 'id_23' 'id_24' 'id_25' 'id_26' 'id_27' 'id_28'\n 'id_29' 'id_30' 'id_31' 'id_33' 'id_34' 'id_35' 'id_36' 'id_37' 'id_38']

SyntaxError: unexpected character after line continuation character (<ipython-input-10-40ee82039d65>, line 1)